In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold

from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

#Pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#Import Braindecode Model EEG Conformer
from braindecode.models import EEGConformer

import gc

/user/home/ko20929/miniconda3/envs/sktime_latest/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
file_list = [ ".ipynb_checkpoints/",
    "Execute New Experiments/Deep Learning/.ipynb_checkpoints/Notebook_25_Conformer_on_my_data_run_on_gpu_main-Copy_more_memory_efficient_training_loop-checkpoint.ipynb",
    "Execute New Experiments/Deep Learning/.ipynb_checkpoints/Notebook_26_Test_Kernel_Dying-checkpoint.ipynb",
    "Execute New Experiments/Deep Learning/.ipynb_checkpoints/Notebook_27_GPU_memory_usage_stats_analysis-checkpoint.ipynb",
    "Execute New Experiments/Deep Learning/.ipynb_checkpoints/test_fig-checkpoint.png",
    "Execute New Experiments/Deep Learning/GPU_Memory_Usage_Stats .xlsx",
    "Execute New Experiments/Deep Learning/N1_20_channel_normalised_dataframes.pkl",
    "Execute New Experiments/Deep Learning/N2_14_channel_normalised_dataframes.pkl",
    "Execute New Experiments/Deep Learning/N2_20_channel_normalised_dataframes.pkl",
    "Execute New Experiments/Deep Learning/N3_20_channel_normalised_dataframes.pkl",
    "Execute New Experiments/Deep Learning/Notebook_25_Conformer_on_my_data_run_on_gpu_main-Copy_more_memory_efficient_training_loop.ipynb",
    "Execute New Experiments/Deep Learning/Notebook_26_Test_Kernel_Dying.ipynb",
    "Execute New Experiments/Deep Learning/Notebook_27_GPU_memory_usage_stats_analysis.ipynb",
    "Execute New Experiments/Deep Learning/REM_20_channel_normalised_dataframes.pkl",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/N1_Conformer_window_120.0_secs_14_channels-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/N2_Conformer_atth5_attd3_120.0_secs_14_channels-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/N2_Conformer_atth5_attd3_60.0_secs_14_channels-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/Wake_Conformer_window_120.0_secs_14_channels-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/Wake_Conformer_window_60.0_secs_14_channels-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/info-checkpoint.txt",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/script_generatedN1_Conformer_atth5_attd3_15.0_secs_14_channelsb_64-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/script_generatedN1_Conformer_atth5_attd3_30.0_secs_14_channelsb_64-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/script_generatedN1_Conformer_atth5_attd3_60.0_secs_14_channelsb_32-checkpoint.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/N1_Conformer_window_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/N1_Conformer_window_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/N2_Conformer_atth5_attd3_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/N2_Conformer_atth5_attd3_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/N3_Conformer_atth5_attd3_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/N3_Conformer_atth5_attd3_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/REM_Conformer_atth5_attd3_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/REM_Conformer_atth5_attd3_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/Wake_Conformer_window_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/Wake_Conformer_window_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/info.txt",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth5_attd3_15.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth5_attd3_15.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth5_attd3_30.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth5_attd3_30.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth5_attd3_60.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth8_attd6_15.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth8_attd6_30.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN1_Conformer_atth8_attd6_60.0_secs_14_channelsb_16.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth5_attd3_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth5_attd3_15.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth5_attd3_15.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth5_attd3_30.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth5_attd3_30.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth5_attd3_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth8_attd6_15.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN2_Conformer_atth8_attd6_30.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth5_attd3_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth5_attd3_15.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth5_attd3_15.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth5_attd3_30.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth5_attd3_30.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth5_attd3_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth8_attd6_15.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedN3_Conformer_atth8_attd6_30.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth5_attd3_120.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth5_attd3_15.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth5_attd3_15.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth5_attd3_30.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth5_attd3_30.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth5_attd3_60.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth8_attd6_15.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedREM_Conformer_atth8_attd6_30.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedWake_Conformer_atth5_attd3_15.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedWake_Conformer_atth5_attd3_15.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedWake_Conformer_atth5_attd3_30.0_secs_14_channels.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedWake_Conformer_atth5_attd3_30.0_secs_14_channelsb_64.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedWake_Conformer_atth8_attd6_15.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Results/Folder_3/script_generatedWake_Conformer_atth8_attd6_30.0_secs_14_channelsb_32.png",
    "Execute New Experiments/Deep Learning/Wake_20_channel_normalised_dataframes.pkl",
    "Execute New Experiments/Deep Learning/execute_conformer.py",
    "Execute New Experiments/Deep Learning/execute_conformer_2.py",
    "Execute New Experiments/Deep Learning/test_fig.png",
    "Execute New Experiments/New_datasets/",
    "Test_BC4_notebook.ipynb" ]   

In [6]:
len(file_list)

78

In [12]:
pngs = [file_name for file_name in file_list if  file_name.endswith(".png")]
pngs

['Execute New Experiments/Deep Learning/.ipynb_checkpoints/test_fig-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/N1_Conformer_window_120.0_secs_14_channels-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/N2_Conformer_atth5_attd3_120.0_secs_14_channels-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/N2_Conformer_atth5_attd3_60.0_secs_14_channels-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/Wake_Conformer_window_120.0_secs_14_channels-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/Wake_Conformer_window_60.0_secs_14_channels-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb_checkpoints/script_generatedN1_Conformer_atth5_attd3_15.0_secs_14_channelsb_64-checkpoint.png',
 'Execute New Experiments/Deep Learning/Results/Folder_3/.ipynb